In [ ]:
# GENERATE CAUDATE TEMPLATE WITH THIS:
#/Applications/workbench/bin_macosx64/wb_command -cifti-label-to-roi Gordon_333_Parcel+FS_Anatomical_Parcel.dlabel.nii caudate_right.dscalar.nii -name caudate_R

#Do it for each structure

#transform these dscalar files to nifti volume files with this:
#/Applications/workbench/bin_macosx64/wb_command -cifti-separate caudate_left.dscalar.nii COLUMN -volume-all caudate_left.nii
#/Applications/workbench/bin_macosx64/wb_command -cifti-separate caudate_right.dscalar.nii COLUMN -volume-all caudate_right.nii


#add these structures:
#fslmaths caudate_left.nii -add caudate_right.nii caudate_bilateral.nii
#gzip -d caudate_bilateral.nii.gz

#Restrict a random cope1 file to follow this structure:
#/Applications/workbench/bin_macosx64/wb_command -cifti-restrict-dense-map cope1.dtseries.nii COLUMN cope1_caudate_bilateral.dtseries.nii -vol-roi caudate_bilateral.nii

#Merge the cope1 files as many times as required:
#/Applications/workbench/bin_macosx64/wb_command -cifti-merge mergedfile8_caudate_bilateral.dtseries.nii -cifti cope1_caudate_bilateral.dtseries.nii -cifti cope1_caudate_bilateral.dtseries.nii -cifti cope1_caudate_bilateral.dtseries.nii -cifti cope1_caudate_bilateral.dtseries.nii -cifti cope1_caudate_bilateral.dtseries.nii -cifti cope1_caudate_bilateral.dtseries.nii -cifti cope1_caudate_bilateral.dtseries.nii -cifti cope1_caudate_bilateral.dtseries.nii

#Transform it to a dscalar file, which is the format that I will need to use to generate the gradient files:
#/Applications/workbench/bin_macosx64/wb_command -cifti-change-mapping mergedfile8_caudate_bilateral.dtseries.nii ROW mergedfile8_caudate_bilateral.dscalar.nii -scalar -name-file dscalar_names.txt
## dscalar_names.txt has one number per row, for the total of gradients, starting with 1.


In [14]:
cd /om/project/UCT1/GRADIENTPROJECT/GRADIENT_PROJECT/python_notebooks_final/cerebellumgradient_initialfiles

/om/project/UCT1/GRADIENTPROJECT/GRADIENT_PROJECT/python_notebooks_final/cerebellumgradient_initialfiles


In [15]:
import numpy as np
import nibabel as nib
from sklearn.metrics import pairwise_distances
sys.path.append("/om/user/xaviergp/GRADIENT_PROJECT/mapalign")
from mapalign import embed
dcon = nib.load('HCP_S1200_1003_rfMRI_MSMAll_groupPCA_d4500ROW_zcorr.dconn.nii').get_data()

In [3]:
#(0, 0, 'CIFTI_STRUCTURE_CORTEX_LEFT')
#(1, 29696, 'CIFTI_STRUCTURE_CORTEX_RIGHT')
#(2, 59412, 'CIFTI_STRUCTURE_ACCUMBENS_LEFT')
#(3, 59547, 'CIFTI_STRUCTURE_ACCUMBENS_RIGHT')
#(4, 59687, 'CIFTI_STRUCTURE_AMYGDALA_LEFT')
#(5, 60002, 'CIFTI_STRUCTURE_AMYGDALA_RIGHT')
#(6, 60334, 'CIFTI_STRUCTURE_BRAIN_STEM')
#(7, 63806, 'CIFTI_STRUCTURE_CAUDATE_LEFT')
#(8, 64534, 'CIFTI_STRUCTURE_CAUDATE_RIGHT')
#(9, 65289, 'CIFTI_STRUCTURE_CEREBELLUM_LEFT')
#(10, 73998, 'CIFTI_STRUCTURE_CEREBELLUM_RIGHT')
#(11, 83142, 'CIFTI_STRUCTURE_DIENCEPHALON_VENTRAL_LEFT')
#(12, 83848, 'CIFTI_STRUCTURE_DIENCEPHALON_VENTRAL_RIGHT')
#(13, 84560, 'CIFTI_STRUCTURE_HIPPOCAMPUS_LEFT')
#(14, 85324, 'CIFTI_STRUCTURE_HIPPOCAMPUS_RIGHT')
#(15, 86119, 'CIFTI_STRUCTURE_PALLIDUM_LEFT')
#(16, 86416, 'CIFTI_STRUCTURE_PALLIDUM_RIGHT')
#(17, 86676, 'CIFTI_STRUCTURE_PUTAMEN_LEFT')
#(18, 87736, 'CIFTI_STRUCTURE_PUTAMEN_RIGHT')
#(19, 88746, 'CIFTI_STRUCTURE_THALAMUS_LEFT')
#(20, 90034, 'CIFTI_STRUCTURE_THALAMUS_RIGHT')

In [16]:
#dcon_fromcerebellumtoleftcaudate = dcon[65289:83142, 63806:64534]
#dcon_fromcerebellumtorightcaudate = dcon[65289:83142, 64534:65289]
#dcon_fromcerebellumtoleftputamen = dcon[65289:83142, 86676:87736]
#dcon_fromcerebellumtorightputamen = dcon[65289:83142, 87736:88746]
#dcon_fromcerebellumtoleftpallidum = dcon[65289:83142, 86119:86416]
#dcon_fromcerebellumtorightpallidum = dcon[65289:83142, 86416:86676]
#dcon_fromcerebellumtoleftaccumbens = dcon[65289:83142, 59412:59547]
#dcon_fromcerebellumtorightaccumbens = dcon[65289:83142, 59547:59687]
#dconn_striatum = np.concatenate((dcon_fromcerebellumtoleftcaudate, dcon_fromcerebellumtorightcaudate, dcon_fromcerebellumtoleftputamen, dcon_fromcerebellumtorightputamen, dcon_fromcerebellumtoleftpallidum, dcon_fromcerebellumtorightpallidum, dcon_fromcerebellumtoleftaccumbens, dcon_fromcerebellumtorightaccumbens), axis = 1)
#dcon = dconn_striatum

dcon_caudate = dcon[63806:65289, 63806:65289]
dcon_putamen = dcon[86676:88746, 86676:88746]
dcon_pallidum = dcon[86119:86676, 86119:86676]
# SELECT HERE THE STRUCTURE THAT WILL BE USED
dcon = dcon_caudate





In [15]:
#dcon = dcon.T
#dcon_thalamus_left = dcon[88746:90033]
#dcon_thalamus_left = dcon_thalamus_left.T

In [17]:
# Generate .dscalar file with the 8 gradients
import numpy as np
import nibabel as nib
from sklearn.metrics import pairwise_distances
dcon = np.tanh(dcon) ## this step takes a while
N = dcon.shape[0]
perc = np.array([np.percentile(x, 90) for x in dcon])
for i in range(dcon.shape[0]):
    print "Row %d" % i
    dcon[i, dcon[i,:] < perc[i]] = 0
dcon[dcon < 0] = 0
aff = 1 - pairwise_distances(dcon, metric = 'cosine')
np.save('cosine_affinity_caudatebilateral.npy', aff)
aff = np.load('cosine_affinity_caudatebilateral.npy')
import sys
emb, res = embed.compute_diffusion_map(aff, alpha = 0.5)
np.save('embedding_dense_caudatebilateral_emb.npy', emb)
np.save('embedding_dense_caudatebilateral_res.npy', res)

res = np.load('embedding_dense_caudatebilateral_res.npy').item()
a = [res['vectors'][:,i]/ res['vectors'][:,0] for i in range(9)]
emb = np.array(a)[1:,:].T
len(emb)
res = nib.load('hcp.tmp.lh.dscalar.nii').get_data()
cortL = np.squeeze(np.array(np.where(res != 0) [0], dtype=np.int32))
res = nib.load('hcp.tmp.rh.dscalar.nii').get_data()
cortR = np.squeeze(np.array(np.where(res != 0) [0], dtype=np.int32))
corLen = len(cortL) + len(cortR)
del res
emb = np.load('embedding_dense_caudatebilateral_emb.npy')
emb.shape
###### have to then run "emb.shape", and generate the merged file for the following steps with the number of maps indicated in the emb.shape


Row 0
Row 1
Row 2
Row 3
Row 4
Row 5
Row 6
Row 7
Row 8
Row 9
Row 10
Row 11
Row 12
Row 13
Row 14
Row 15
Row 16
Row 17
Row 18
Row 19
Row 20
Row 21
Row 22
Row 23
Row 24
Row 25
Row 26
Row 27
Row 28
Row 29
Row 30
Row 31
Row 32
Row 33
Row 34
Row 35
Row 36
Row 37
Row 38
Row 39
Row 40
Row 41
Row 42
Row 43
Row 44
Row 45
Row 46
Row 47
Row 48
Row 49
Row 50
Row 51
Row 52
Row 53
Row 54
Row 55
Row 56
Row 57
Row 58
Row 59
Row 60
Row 61
Row 62
Row 63
Row 64
Row 65
Row 66
Row 67
Row 68
Row 69
Row 70
Row 71
Row 72
Row 73
Row 74
Row 75
Row 76
Row 77
Row 78
Row 79
Row 80
Row 81
Row 82
Row 83
Row 84
Row 85
Row 86
Row 87
Row 88
Row 89
Row 90
Row 91
Row 92
Row 93
Row 94
Row 95
Row 96
Row 97
Row 98
Row 99
Row 100
Row 101
Row 102
Row 103
Row 104
Row 105
Row 106
Row 107
Row 108
Row 109
Row 110
Row 111
Row 112
Row 113
Row 114
Row 115
Row 116
Row 117
Row 118
Row 119
Row 120
Row 121
Row 122
Row 123
Row 124
Row 125
Row 126
Row 127
Row 128
Row 129
Row 130
Row 131
Row 132
Row 133
Row 134
Row 135
Row 136
Row 137
Row 13

pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1


(1483, 8)

In [18]:
emb.shape

(1483, 8)

In [19]:
tmp = nib.load('mergedfile8_caudate_bilateral.dscalar.nii')
tmp_cifti = nib.cifti2.load('mergedfile8_caudate_bilateral.dscalar.nii')
data = tmp_cifti.get_data() * 0
mim = tmp.header.matrix[1]
for idx, bm in enumerate(mim.brain_models):
    print ((idx, bm.index_offset, bm.brain_structure))
img = nib.cifti2.Cifti2Image(emb.T, nib.cifti2.Cifti2Header(tmp.header.matrix))
img.to_filename('result_caudatebilateral.dscalar.nii')

(0, 0, 'CIFTI_STRUCTURE_CAUDATE_LEFT')
(1, 728, 'CIFTI_STRUCTURE_CAUDATE_RIGHT')
